In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import datetime
import glob
from google.colab import files
import numpy as np
import os
import pandas as pd

In [ ]:
year=[2020]
for yearss in year:
    # Lista de meses
    i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))
    year=yearss
    # Descargar y extraer archivos mensuales
    for year in [year]:
        for month, month_name in i2m:
            # Descargar archivo comprimido
            os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            # Extraer archivos del formato 7z
            os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            # Eliminar archivo comprimido
            os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

    # Obtener lista de archivos CSV en el directorio actual
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-3:] == "csv"]
    listCsv=sorted(listCsv)

    # Procesar y transformar archivos CSV
    for i in range(0, 12, 1):
        # Cargar archivo CSV en DataFrame
        ftemp = pd.read_csv(listCsv[i])

        # Convertir columnas de fecha y hora a objetos datetime
        ftemp['last_reported'] = pd.to_datetime(ftemp['last_reported'], unit='s')
        ftemp['last_updated'] = pd.to_datetime(ftemp['last_updated'], unit='s')

        # Extraer información temporal
        ftemp['year'] = ftemp.last_reported.dt.year
        ftemp['month'] = ftemp.last_reported.dt.month
        ftemp = ftemp[ftemp['month'] == i+1]
        ftemp['hour'] = ftemp.last_reported.dt.hour
        ftemp['dayyear'] = ftemp.last_reported.dt.day_of_year
        ftemp['houryear'] = (ftemp.dayyear - 1) * 24 + ftemp.hour

        # Calcular la suma de las columnas "num_docks_available" y "num_bikes_available" en una nueva columna "Llocs", son la suma de bicis y sitios bacios
        ftemp['Llocs'] = ftemp['num_docks_available'] + ftemp['num_bikes_available']
        ftemp.dropna(subset=['num_docks_available', 'Llocs'], inplace=True)
        ftemp = ftemp[ftemp["num_docks_available"] >= 0]
        ftemp = ftemp[ftemp["num_docks_available"] <= 60]
        ftemp = ftemp[ftemp["Llocs"]  <= 60]
        ftemp = ftemp[ftemp["num_docks_available"]  <= ftemp["Llocs"]]
        # Configurar columna "last_reported" como índice, es la fecha de la fila
        ftemp = ftemp.set_index('last_reported')

        # Agrupar por "station_id" y "houryear" y calcular la media de los valores para cada grupo
        ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
        ftemp = ftemp.reset_index()

        # Convertir columnas "houryear" y "year" a enteros
        ftemp['houryear'] = ftemp['houryear'].astype(int)
        ftemp['year'] = ftemp['year'].astype(int)

        # Calcular fecha y hora como una cadena con formato 'AAAA/MM/DD HH:MM:SS'
        ftemp['time'] = pd.to_datetime(ftemp['year'].astype(str), format='%Y') + pd.to_timedelta(ftemp['houryear'], unit='h')
        ftemp['time'] = ftemp['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
        ftemp['dayofweek'] = pd.to_datetime(ftemp['time']).dt.dayofweek
        ftemp['time2'] = pd.to_datetime(ftemp['time']).dt.date

        # Guardar DataFrame en archivo CSV
        ftemp.to_csv(str(year)+ str(i + 1) + 'A.csv', index=False)

        # Descargar archivo (posiblemente específico de un entorno o biblioteca particular)
        # files.download('/content/2022' + str(i + 1) + '.csv')
        # Liberar memoria eliminando el DataFrame
        del ftemp
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-5:] == "A.csv"]
    # Procesar y transformar archivos CSV
    for i in range(0, len(listCsv), 1):
        # Cargar archivo CSV en DataFrame
        ftemp = pd.read_csv(listCsv[i])
        # Abrimos el archivo de precipitacion
        df = pd.read_csv('/content/drive/MyDrive/prova/Precipitacio.csv')
        # Eliminamos las columnas no necesarias y creamos unas con variables de tiempo
        df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
        df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
        df['year'] = df.time.dt.year
        df['dayyear'] = df.time.dt.day_of_year
        # Creamos una variable que indique si ha llovido ese dia ponga un 1, indicando lluvia en ese dia
        df = df.groupby(['year', 'dayyear']).sum()
        df['Rain'] = np.where(df['VALOR_LECTURA'] > 1, 1, 0)
        # Rain_Lectura será el valor real de la lectura de lluvia en esa hora
        df['Rain_Lectura'] = df['VALOR_LECTURA']
        df = df.drop(['VALOR_LECTURA'], axis=1)
        df = df.reset_index()
        ftemp['time2'] = pd.to_datetime(ftemp['time2'])
        df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
        # Ajuntamos la informacion del csv original con el de precipitacion
        Total = pd.merge_ordered(ftemp, df, how='left', on=['time2'], fill_method='ffill')
        Total = Total.drop(['year_y', 'dayyear_y'], axis=1)
        # Realizamos lo mismo con los valores de viento, en ese caso si el viento es superior a 20 m/s en algun momento
        # se rellena la variable WIND con un 1 todo el dia para in
        df = pd.read_csv('/content/drive/MyDrive/prova/Vent.csv')
        df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
        df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
        df['year'] = df.time.dt.year
        df['dayyear'] = df.time.dt.day_of_year
        df = df.groupby(['year', 'dayyear']).max()
        df['Wind'] = np.where(df['VALOR_LECTURA'] > 20, 1, 0)
        df['Wind_Lectura'] = df['VALOR_LECTURA']
        df = df.drop(['VALOR_LECTURA'], axis=1)
        df = df.drop(['DATA_LECTURA', 'time'], axis=1)
        df = df.reset_index()
        df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
        Total = pd.merge_ordered(Total, df, how='left', on=['time2'], fill_method='ffill')
        Total.to_csv(listCsv[i][:-5]+'.csv', index=False)
        # Descargar archivo (si estás en un entorno que permite la descarga)
        files.download(listCsv[i][:-5]+'.csv')


<ipython-input-4-63b10b51fe24>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-4-63b10b51fe24>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-4-63b10b51fe24>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id',

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-4-63b10b51fe24>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-4-63b10b51fe24>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-3:] == "csv"]
    listCsv=sorted(listCsv)
    # Eliminar cada archivo en la lista
    for file_name in listCsv:
        os.remove(file_name)

In [ ]:
year=[2021]
for yearss in year:
    # Lista de meses
    i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))
    year=yearss
    # Descargar y extraer archivos mensuales
    for year in [year]:
        for month, month_name in i2m:
            # Descargar archivo comprimido
            os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            # Extraer archivos del formato 7z
            os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            # Eliminar archivo comprimido
            os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

    # Obtener lista de archivos CSV en el directorio actual
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-3:] == "csv"]
    listCsv=sorted(listCsv)

    # Procesar y transformar archivos CSV
    for i in range(0, 12, 1):
        # Cargar archivo CSV en DataFrame
        ftemp = pd.read_csv(listCsv[i])

        # Convertir columnas de fecha y hora a objetos datetime
        ftemp['last_reported'] = pd.to_datetime(ftemp['last_reported'], unit='s')
        ftemp['last_updated'] = pd.to_datetime(ftemp['last_updated'], unit='s')

        # Extraer información temporal
        ftemp['year'] = ftemp.last_reported.dt.year
        ftemp['month'] = ftemp.last_reported.dt.month
        ftemp = ftemp[ftemp['month'] == i+1]
        ftemp['hour'] = ftemp.last_reported.dt.hour
        ftemp['dayyear'] = ftemp.last_reported.dt.day_of_year
        ftemp['houryear'] = (ftemp.dayyear - 1) * 24 + ftemp.hour

        # Calcular la suma de las columnas "num_docks_available" y "num_bikes_available" en una nueva columna "Llocs", son la suma de bicis y sitios bacios
        ftemp['Llocs'] = ftemp['num_docks_available'] + ftemp['num_bikes_available']
        ftemp.dropna(subset=['num_docks_available', 'Llocs'], inplace=True)
        ftemp = ftemp[ftemp["num_docks_available"] >= 0]
        ftemp = ftemp[ftemp["num_docks_available"] <= 60]
        ftemp = ftemp[ftemp["Llocs"]  <= 60]
        ftemp = ftemp[ftemp["num_docks_available"]  <= ftemp["Llocs"]]
        # Configurar columna "last_reported" como índice, es la fecha de la fila
        ftemp = ftemp.set_index('last_reported')

        # Agrupar por "station_id" y "houryear" y calcular la media de los valores para cada grupo
        ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
        ftemp = ftemp.reset_index()

        # Convertir columnas "houryear" y "year" a enteros
        ftemp['houryear'] = ftemp['houryear'].astype(int)
        ftemp['year'] = ftemp['year'].astype(int)

        # Calcular fecha y hora como una cadena con formato 'AAAA/MM/DD HH:MM:SS'
        ftemp['time'] = pd.to_datetime(ftemp['year'].astype(str), format='%Y') + pd.to_timedelta(ftemp['houryear'], unit='h')
        ftemp['time'] = ftemp['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
        ftemp['dayofweek'] = pd.to_datetime(ftemp['time']).dt.dayofweek
        ftemp['time2'] = pd.to_datetime(ftemp['time']).dt.date

        # Guardar DataFrame en archivo CSV
        ftemp.to_csv(str(year)+ str(i + 1) + 'A.csv', index=False)

        # Descargar archivo (posiblemente específico de un entorno o biblioteca particular)
        # files.download('/content/2022' + str(i + 1) + '.csv')
        # Liberar memoria eliminando el DataFrame
        del ftemp
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-5:] == "A.csv"]
    # Procesar y transformar archivos CSV
    for i in range(0, len(listCsv), 1):
        # Cargar archivo CSV en DataFrame
        ftemp = pd.read_csv(listCsv[i])
        # Abrimos el archivo de precipitacion
        df = pd.read_csv('/content/drive/MyDrive/prova/Precipitacio.csv')
        # Eliminamos las columnas no necesarias y creamos unas con variables de tiempo
        df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
        df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
        df['year'] = df.time.dt.year
        df['dayyear'] = df.time.dt.day_of_year
        # Creamos una variable que indique si ha llovido ese dia ponga un 1, indicando lluvia en ese dia
        df = df.groupby(['year', 'dayyear']).sum()
        df['Rain'] = np.where(df['VALOR_LECTURA'] > 1, 1, 0)
        # Rain_Lectura será el valor real de la lectura de lluvia en esa hora
        df['Rain_Lectura'] = df['VALOR_LECTURA']
        df = df.drop(['VALOR_LECTURA'], axis=1)
        df = df.reset_index()
        ftemp['time2'] = pd.to_datetime(ftemp['time2'])
        df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
        # Ajuntamos la informacion del csv original con el de precipitacion
        Total = pd.merge_ordered(ftemp, df, how='left', on=['time2'], fill_method='ffill')
        Total = Total.drop(['year_y', 'dayyear_y'], axis=1)
        # Realizamos lo mismo con los valores de viento, en ese caso si el viento es superior a 20 m/s en algun momento
        # se rellena la variable WIND con un 1 todo el dia para in
        df = pd.read_csv('/content/drive/MyDrive/prova/Vent.csv')
        df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
        df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
        df['year'] = df.time.dt.year
        df['dayyear'] = df.time.dt.day_of_year
        df = df.groupby(['year', 'dayyear']).max()
        df['Wind'] = np.where(df['VALOR_LECTURA'] > 20, 1, 0)
        df['Wind_Lectura'] = df['VALOR_LECTURA']
        df = df.drop(['VALOR_LECTURA'], axis=1)
        df = df.drop(['DATA_LECTURA', 'time'], axis=1)
        df = df.reset_index()
        df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
        Total = pd.merge_ordered(Total, df, how='left', on=['time2'], fill_method='ffill')
        Total.to_csv(listCsv[i][:-5]+'.csv', index=False)
        # Descargar archivo (si estás en un entorno que permite la descarga)
        files.download(listCsv[i][:-5]+'.csv')


<ipython-input-3-fd5af0525b8b>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-3-fd5af0525b8b>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-3-fd5af0525b8b>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id',

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-3-fd5af0525b8b>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-3-fd5af0525b8b>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-3:] == "csv"]
    listCsv=sorted(listCsv)
    # Eliminar cada archivo en la lista
    for file_name in listCsv:
        os.remove(file_name)

In [ ]:
year=[2022]
for yearss in year:
    # Lista de meses
    i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))
    year=yearss
    # Descargar y extraer archivos mensuales
    for year in [year]:
        for month, month_name in i2m:
            # Descargar archivo comprimido
            os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            # Extraer archivos del formato 7z
            os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
            # Eliminar archivo comprimido
            os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

    # Obtener lista de archivos CSV en el directorio actual
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-3:] == "csv"]
    listCsv=sorted(listCsv)

    # Procesar y transformar archivos CSV
    for i in range(0, 12, 1):
        # Cargar archivo CSV en DataFrame
        ftemp = pd.read_csv(listCsv[i])

        # Convertir columnas de fecha y hora a objetos datetime
        ftemp['last_reported'] = pd.to_datetime(ftemp['last_reported'], unit='s')
        ftemp['last_updated'] = pd.to_datetime(ftemp['last_updated'], unit='s')

        # Extraer información temporal
        ftemp['year'] = ftemp.last_reported.dt.year
        ftemp['month'] = ftemp.last_reported.dt.month
        ftemp = ftemp[ftemp['month'] == i+1]
        ftemp['hour'] = ftemp.last_reported.dt.hour
        ftemp['dayyear'] = ftemp.last_reported.dt.day_of_year
        ftemp['houryear'] = (ftemp.dayyear - 1) * 24 + ftemp.hour

        # Calcular la suma de las columnas "num_docks_available" y "num_bikes_available" en una nueva columna "Llocs", son la suma de bicis y sitios bacios
        ftemp['Llocs'] = ftemp['num_docks_available'] + ftemp['num_bikes_available']
        ftemp.dropna(subset=['num_docks_available', 'Llocs'], inplace=True)
        ftemp = ftemp[ftemp["num_docks_available"] >= 0]
        ftemp = ftemp[ftemp["num_docks_available"] <= 60]
        ftemp = ftemp[ftemp["Llocs"]  <= 60]
        ftemp = ftemp[ftemp["num_docks_available"]  <= ftemp["Llocs"]]
        # Configurar columna "last_reported" como índice, es la fecha de la fila
        ftemp = ftemp.set_index('last_reported')

        # Agrupar por "station_id" y "houryear" y calcular la media de los valores para cada grupo
        ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
        ftemp = ftemp.reset_index()

        # Convertir columnas "houryear" y "year" a enteros
        ftemp['houryear'] = ftemp['houryear'].astype(int)
        ftemp['year'] = ftemp['year'].astype(int)

        # Calcular fecha y hora como una cadena con formato 'AAAA/MM/DD HH:MM:SS'
        ftemp['time'] = pd.to_datetime(ftemp['year'].astype(str), format='%Y') + pd.to_timedelta(ftemp['houryear'], unit='h')
        ftemp['time'] = ftemp['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
        ftemp['dayofweek'] = pd.to_datetime(ftemp['time']).dt.dayofweek
        ftemp['time2'] = pd.to_datetime(ftemp['time']).dt.date

        # Guardar DataFrame en archivo CSV
        ftemp.to_csv(str(year)+ str(i + 1) + 'A.csv', index=False)

        # Descargar archivo (posiblemente específico de un entorno o biblioteca particular)
        # files.download('/content/2022' + str(i + 1) + '.csv')
        # Liberar memoria eliminando el DataFrame
        del ftemp
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-5:] == "A.csv"]
    # Procesar y transformar archivos CSV
    for i in range(0, len(listCsv), 1):
        # Cargar archivo CSV en DataFrame
        ftemp = pd.read_csv(listCsv[i])
        # Abrimos el archivo de precipitacion
        df = pd.read_csv('/content/drive/MyDrive/prova/Precipitacio.csv')
        # Eliminamos las columnas no necesarias y creamos unas con variables de tiempo
        df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
        df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
        df['year'] = df.time.dt.year
        df['dayyear'] = df.time.dt.day_of_year
        # Creamos una variable que indique si ha llovido ese dia ponga un 1, indicando lluvia en ese dia
        df = df.groupby(['year', 'dayyear']).sum()
        df['Rain'] = np.where(df['VALOR_LECTURA'] > 1, 1, 0)
        # Rain_Lectura será el valor real de la lectura de lluvia en esa hora
        df['Rain_Lectura'] = df['VALOR_LECTURA']
        df = df.drop(['VALOR_LECTURA'], axis=1)
        df = df.reset_index()
        ftemp['time2'] = pd.to_datetime(ftemp['time2'])
        df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
        # Ajuntamos la informacion del csv original con el de precipitacion
        Total = pd.merge_ordered(ftemp, df, how='left', on=['time2'], fill_method='ffill')
        Total = Total.drop(['year_y', 'dayyear_y'], axis=1)
        # Realizamos lo mismo con los valores de viento, en ese caso si el viento es superior a 20 m/s en algun momento
        # se rellena la variable WIND con un 1 todo el dia para in
        df = pd.read_csv('/content/drive/MyDrive/prova/Vent.csv')
        df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
        df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
        df['year'] = df.time.dt.year
        df['dayyear'] = df.time.dt.day_of_year
        df = df.groupby(['year', 'dayyear']).max()
        df['Wind'] = np.where(df['VALOR_LECTURA'] > 20, 1, 0)
        df['Wind_Lectura'] = df['VALOR_LECTURA']
        df = df.drop(['VALOR_LECTURA'], axis=1)
        df = df.drop(['DATA_LECTURA', 'time'], axis=1)
        df = df.reset_index()
        df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
        Total = pd.merge_ordered(Total, df, how='left', on=['time2'], fill_method='ffill')
        Total.to_csv(listCsv[i][:-5]+'.csv', index=False)
        # Descargar archivo (si estás en un entorno que permite la descarga)
        files.download(listCsv[i][:-5]+'.csv')


<ipython-input-5-ef145664883b>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-5-ef145664883b>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-5-ef145664883b>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id',

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-5-ef145664883b>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-5-ef145664883b>:83: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
    fileList = os.listdir()
    listCsv = [file for file in fileList if file[-3:] == "csv"]
    listCsv=sorted(listCsv)
    # Eliminar cada archivo en la lista
    for file_name in listCsv:
        os.remove(file_name)

In [ ]:
# Lista de meses
i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))
year=2019
# Descargar y extraer archivos mensuales
for year in [year]:
    for month, month_name in i2m:
        # Descargar archivo comprimido
        os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        # Extraer archivos del formato 7z
        os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        # Eliminar archivo comprimido
        os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

# Obtener lista de archivos CSV en el directorio actual
fileList = os.listdir()
listCsv = [file for file in fileList if file[-3:] == "csv"]
listCsv=sorted(listCsv)

# Procesar y transformar archivos CSV
for i in range(0, 10, 1):
    # Cargar archivo CSV en DataFrame
    ftemp = pd.read_csv(listCsv[i])

    # Convertir columnas de fecha y hora a objetos datetime
    ftemp['last_reported'] = pd.to_datetime(ftemp['last_reported'], unit='s')
    ftemp['last_updated'] = pd.to_datetime(ftemp['last_updated'], unit='s')

    # Extraer información temporal
    ftemp['year'] = ftemp.last_reported.dt.year
    ftemp['month'] = ftemp.last_reported.dt.month
    ftemp = ftemp[ftemp['month'] == i+3]
    ftemp['hour'] = ftemp.last_reported.dt.hour
    ftemp['dayyear'] = ftemp.last_reported.dt.day_of_year
    ftemp['houryear'] = (ftemp.dayyear - 1) * 24 + ftemp.hour

    # Calcular la suma de las columnas "num_docks_available" y "num_bikes_available" en una nueva columna "Llocs", son la suma de bicis y sitios bacios
    ftemp['Llocs'] = ftemp['num_docks_available'] + ftemp['num_bikes_available']
    ftemp.dropna(subset=['num_docks_available', 'Llocs'], inplace=True)
    ftemp = ftemp[ftemp["num_docks_available"] >= 0]
    ftemp = ftemp[ftemp["num_docks_available"] <= 60]
    ftemp = ftemp[ftemp["Llocs"]  <= 60]
    ftemp = ftemp[ftemp["num_docks_available"]  <= ftemp["Llocs"]]
    # Configurar columna "last_reported" como índice, es la fecha de la fila
    ftemp = ftemp.set_index('last_reported')

    # Agrupar por "station_id" y "houryear" y calcular la media de los valores para cada grupo
    ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
    ftemp = ftemp.reset_index()

    # Convertir columnas "houryear" y "year" a enteros
    ftemp['houryear'] = ftemp['houryear'].astype(int)
    ftemp['year'] = ftemp['year'].astype(int)

    # Calcular fecha y hora como una cadena con formato 'AAAA/MM/DD HH:MM:SS'
    ftemp['time'] = pd.to_datetime(ftemp['year'].astype(str), format='%Y') + pd.to_timedelta(ftemp['houryear'], unit='h')
    ftemp['time'] = ftemp['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
    ftemp['dayofweek'] = pd.to_datetime(ftemp['time']).dt.dayofweek
    ftemp['time2'] = pd.to_datetime(ftemp['time']).dt.date

    # Guardar DataFrame en archivo CSV
    ftemp.to_csv(str(year)+ str(i + 3) + 'A.csv', index=False)

    # Descargar archivo (posiblemente específico de un entorno o biblioteca particular)
    # files.download('/content/2022' + str(i + 1) + '.csv')
    # Liberar memoria eliminando el DataFrame
    del ftemp
fileList = os.listdir()
listCsv = [file for file in fileList if file[-5:] == "A.csv"]
# Procesar y transformar archivos CSV
for i in range(0, len(listCsv), 1):
    # Cargar archivo CSV en DataFrame
    ftemp = pd.read_csv(listCsv[i])
    # Abrimos el archivo de precipitacion
    df = pd.read_csv('/content/drive/MyDrive/prova/Precipitacio.csv')
    # Eliminamos las columnas no necesarias y creamos unas con variables de tiempo
    df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
    df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
    df['year'] = df.time.dt.year
    df['dayyear'] = df.time.dt.day_of_year
    # Creamos una variable que indique si ha llovido ese dia ponga un 1, indicando lluvia en ese dia
    df = df.groupby(['year', 'dayyear']).sum()
    df['Rain'] = np.where(df['VALOR_LECTURA'] > 1, 1, 0)
    # Rain_Lectura será el valor real de la lectura de lluvia en esa hora
    df['Rain_Lectura'] = df['VALOR_LECTURA']
    df = df.drop(['VALOR_LECTURA'], axis=1)
    df = df.reset_index()
    ftemp['time2'] = pd.to_datetime(ftemp['time2'])
    df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
    # Ajuntamos la informacion del csv original con el de precipitacion
    Total = pd.merge_ordered(ftemp, df, how='left', on=['time2'], fill_method='ffill')
    Total = Total.drop(['year_y', 'dayyear_y'], axis=1)
    # Realizamos lo mismo con los valores de viento, en ese caso si el viento es superior a 20 m/s en algun momento
    # se rellena la variable WIND con un 1 todo el dia para in
    df = pd.read_csv('/content/drive/MyDrive/prova/Vent.csv')
    df = df.drop(['CODI_ESTACIO', 'CODI_BASE'], axis=1)
    df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
    df['year'] = df.time.dt.year
    df['dayyear'] = df.time.dt.day_of_year
    df = df.groupby(['year', 'dayyear']).max()
    df['Wind'] = np.where(df['VALOR_LECTURA'] > 20, 1, 0)
    df['Wind_Lectura'] = df['VALOR_LECTURA']
    df = df.drop(['VALOR_LECTURA'], axis=1)
    df = df.drop(['DATA_LECTURA', 'time'], axis=1)
    df = df.reset_index()
    df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
    Total = pd.merge_ordered(Total, df, how='left', on=['time2'], fill_method='ffill')
    Total.to_csv(listCsv[i][:-5]+'.csv', index=False)
    # Descargar archivo (si estás en un entorno que permite la descarga)
    files.download(listCsv[i][:-5]+'.csv')


<ipython-input-8-69c047fd89b5>:47: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-8-69c047fd89b5>:47: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
<ipython-input-8-69c047fd89b5>:47: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ftemp = ftemp.groupby(['station_id',

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-8-69c047fd89b5>:81: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-8-69c047fd89b5>:81: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['year', 'dayyear']).sum()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
fileList = os.listdir()
listCsv = [file for file in fileList if file[-3:] == "csv"]
listCsv=sorted(listCsv)
# Eliminar cada archivo en la lista
for file_name in listCsv:
    os.remove(file_name)